**目录**

[toc]

## MLND 毕业项目

* 作者：RayZen
* 日期：2018 年 6 月 5 日

### Ⅰ. 问题定义

**1.1 项目概述**

`Rossmann` 是经营了超过 `3,000` 家遍布欧洲 `7` 个国家的公司。现在 `Rossmann` 经营者希望能够预测六周的每日销售情况，而它的销售情况收到多个因素影响，例如：促销、竞争对手、学校和国家节假日、季节性因素以及本地化等因素等的影响。因为根据每家店的管理者根据当地的实际情况进行了销售预测，预测结果的准确度差异太大。

本次项目，`Rossmann` 的管理者希望能够通过提高预测的准确性来帮助他们更有效的做出工作计划，以及同时达到提高效率和机动性。对此，提出了需要增强预测的 **稳定性**，也就是说需要对经营额进行预测，这是一个连续性数据；`data` 是具有 `label` 的数据。从对数据初步探索可以，该数据集包括了 2013 年 1 月 1 日至 2015 年 7 月 31 日共计大约两年半的数据集，从下图可知：

![](../img/moving_average_by_date.png)


**1.2 问题陈述**

本项目中提供的训练数据集由 `train.csv` 提供销售数据信息，`store.csv` 提供营业的商店信息（其中营业时期范围为 2013 年 1 月 1 日至 2015 年 7 月 31 日）。首先需要将两个数据文件进行整合为一个训练数据集。经验证，该数据集是一个具有 17 个 `features` 和 1 个 `label`，而且该 `label` 是一个连续型数据。那么也就证实了，在本项目中需要通过监督式学习来完成对项目分析。

其次，在完成数据的基本探索分析后，需要对数据的某些 `feature` 的数据进行重新构建以及筛选。因为在本数据集的 `features` 大部分都是类别型数据，一方面需要将相关数据进行转换，用以表达相关信息；另一方面，初步探索中已经知道该数据是同时是一个时间序列类型数据，对这类数据在进行数据信息转换的同时需要考虑到时间分析的连续性，可能随机性不适用于该数据类型。

最终的数据结果是能够获得稳定性高的预测测试数据的结果。另一方面，考虑到实际应用层面，希望最终的模型是可以被实际应用到现实中的场景中。例如，将该模型用到 `Rossmann` 的商店营业额预测，以帮助它们建立有效的工作计划，提到工作生产效率和机动性。


**1.3 评价指标**
1. 在基准模型选择方面，可以从几个方面进行考虑：1）从已有的模型中进行选择，这个需要进行先构建一个模型；2）依照实际数据集的数据进行评价，这需要依据每个类型的对象数据值——平均值或者中位数进行对照。但是因为该数据集是一个时间序列的数据集，存在其他潜在因素因素影响，所以不适用直接使用对象数据值来进行评估；3）参考已经完成的一定数据量的模型得分。综合以上特点判断，决定选择使用第三种方式作为模型基准，即以 `kaggle` 竞赛中的前 `10%` 作为基准——得到的测试评分为 `0.11773`。

2. 根据项目发起人的要求，项目最终评价指标是使用`RMSPE`(即：`Root Mean Square Percentage Error`)，其计算公式如下：<img src="https://latex.codecogs.com/gif.latex?{RMSPE=\sqrt{\frac{1}{n}\Sigma_{i=1}^{n}(\frac{y_i-\hat{y_i}}{y_i})^2}}" title="{RMSPE=\sqrt{\frac{1}{n}\Sigma_{i=1}^{n}(\frac{y_i-\hat{y_i}}{y_i})^2}}" align="center"/>
    该指标和另一个常用的评估指标 `MPE`相比， 一方面避免了因方向和绝对大小，导致评估范围过大和计算误导。使用 `RMSPE` 的优势在时间序列上能够比较真实的变化比例，同时也能计算出平均的百分比变化。

### Ⅱ. 分析
**2.1 数据探索**

**2.1.1 异常值探索**

在对数据分析之前，需要将所有需要的数据 `store` 和 `train` 进行整合为一个 `data` 数据集。首先对 `Sales` 和 `Customers` 数据进行异常值分析，从 `Boxplot` 可以看出这两个 `features` 中存在极端值，根据情况需要删除极大值点。

![](../img/boxplot_sales_customers.png)

其中 `Sales` 的极值索引行信息如下：

```
Store                                        909
DayOfWeek                                      1
Date                         2015-06-22 00:00:00
Sales                                      41551
Customers                                   1721
Open                                           1
Promo                                          0
StateHoliday                                   0
SchoolHoliday                                  0
StoreType                                      a
Assortment                                     c
CompetitionDistance                         1680
CompetitionOpenSinceMonth                    NaN
CompetitionOpenSinceYear                     NaN
Promo2                                         1
Promo2SinceWeek                               45
Promo2SinceYear                             2009
PromoInterval                    Feb,May,Aug,Nov
CompetitionOpenDate                          NaT
Name: 44393, dtype: object
```
其中 `Customers` 的极值索引行信息如下：
```
Store                                        817
DayOfWeek                                      2
Date                         2013-01-22 00:00:00
Sales                                      27190
Customers                                   7388
Open                                           1
Promo                                          1
StateHoliday                                   0
SchoolHoliday                                  0
StoreType                                      a
Assortment                                     a
CompetitionDistance                          140
CompetitionOpenSinceMonth                      3
CompetitionOpenSinceYear                    2006
Promo2                                         0
Promo2SinceWeek                              NaN
Promo2SinceYear                              NaN
PromoInterval                                NaN
CompetitionOpenDate          2006-03-01 00:00:00
Name: 993496, dtype: object
```
根据各行的信息判断，`Sales` 的极值并非因为节假日或者促销等原因导致的，所以对这个极值需要进行删除；而 `Customers` 中，因为有促销活动而导致了消费者数量增加，这个事件具有合理性。因此最终删除 `Sales` 的极值行信息。

**2.1.2 销售额和客户数量关系探索**

首先对 `Sales` 和 `Customers` 数据进行探索，使用散点图的方式和统计学分析的方式来对两者的相关性进行分析。首先对两者进行散点图展示，可以看出两者还是有相关性。因此通过统计学模型（其中使用 `OLS` 模型）进行分析，计算出两者的相关性结果。

散点图展示：
![](../img/scatter_scales_customers.png)

相关性分析报告：

```
                   Results: Ordinary least squares
======================================================================
Model:              OLS              Adj. R-squared:     0.930        
Dependent Variable: Sales            AIC:                18173504.6382
Date:               2018-06-13 17:20 BIC:                18173516.4707
No. Observations:   1017208          Log-Likelihood:     -9.0868e+06  
Df Model:           1                F-statistic:        1.355e+07    
Df Residuals:       1017207          Prob (F-statistic): 0.00         
R-squared:          0.930            Scale:              3.3625e+06   
------------------------------------------------------------------------
                Coef.    Std.Err.       t       P>|t|    [0.025   0.975]
------------------------------------------------------------------------
Customers       8.5237     0.0023   3681.1981   0.0000   8.5192   8.5283
----------------------------------------------------------------------
Omnibus:             481000.264     Durbin-Watson:        1.714       
Prob(Omnibus):       0.000          Jarque-Bera (JB):     12544512.549
Skew:                -1.724         Prob(JB):             0.000       
Kurtosis:            19.855         Condition No.:        1           
======================================================================
```
从分析结果可以看出，两者具有相关性具有显著性，并且 $R^2=0.930$，具有强相关性

**2.1.3 每周销售额度和客户数量分析**

自 2013 年至 2015 年每周平均销售额度和平均客户数量分析可以得出以下结论：

1. 在星期天来商店购物的人数明显偏少，同时营业额也是偏少
2. 在星期一、星期二以及星期五购物的人数和营业额是较高的，其中在星期一的营业额和客户数量最高
3. 对星期天营业商店进行计数统计，其中关闭的商店占了绝大多数。因此因为营业状态的差异，在进行平均值计算的时候出现不符合实际情况

在每年中每周平均销售额度：
![](../img/average_customers_every_year_week.png)
在每年中每周平均客户数量：
![](../img/average_customers_every_year_week.png)
在每年星期天中商店营业与否统计表格：

||2013|2014|2015|
|:---:|:---:|:---:|:---|
|Closed|56584|51925|32628|
|Open|1396|1375|822|


**2.1.4 每月销售额度和客户数量分析**

自 2013 年至 2015 年每月平均销售额度和平均客户数量分析可以得出以下结论：

1. 每月同比统计平均销售额和平均用户数量，随着年份增加，该额度也整体表现增高的趋势
2. 年度平均销售额度和平均用户数量最高月份是在 11 月份和 12 月份，其次是在 6 月份和 7 月份，以及 3 月份

每月平均销售额在每年中变化：
![](../img/average_sales_every_month.png)

每月平均用户数量在每年中变化：
![](../img/average_customers_every_month.png)

**2.1.5 促销对每月销售额度和客户数量影响分析**

自 2013 年至 2015 年，是否有进行促销活动对每月平均销售额度和平均客户数量影响分析可以得出以下结论:促销对月平均销售额和平均客户数量，都有正面的影响。即促销活动促进了销售额和用户数量的增加；在 12 月份中影响表现极其明显，这可能是因为不仅是促销的影响

促销对平均销售额度的影响：
![](../img/promotion_with_sales.png)

促销对平均用户数量的影响：
![](../img/promotion_with_customers.png)



**2.1.6 商店类型和商店经营策略差异分析**

1. 四种商店类型中, B 类商店的平均营业额是最高的，同时光顾的客户数量也是最多的。这样可能会导致单位客户销售额的平均值降低，最后从单位客户销售额的分析中体现了出来。对 B 类商店的客户猜测，他们主要是购买小宗药品以及不是集中购买
2. 其他三类商店的每月的平均营业额差异不大，另外在光顾商店的客户数量差异也不是很大
3. 分析每月中单位客户销售额的平均值来看，A 类商店是最高的，其次是 D 类，反而在 B 类商店是最低的
4. 商店经营策略差异方面，对 B 类商店影响比较大，其中采用 `extended` 方式经营的 B 类商店每月平均客户数量和平均销售额是最高的。对其他类型的商店，对平局客户数量和平均销售额影响不明显

不同商店类型和经营策略的商店每月平均销售额对比：

![](../img/storetype_sales.png)

不同商店类型和经营策略的商店每月平均客户数量对比：

![](../img/storetype_customers.png)

不同商店类型和经营策略的商店每月单位客户销售额的对比：

![](../img/storetype_salespercustomers.png)